# LEOHS (Landsat ETM+ OLI Harmonization Script)
This is still being developed, Coded by Galen Richardson

In [1]:
import geopandas as gpd #geospatial libraries
from shapely.geometry import Point, box
import rasterio,ee,geemap
from rasterio.features import geometry_mask
from geopy.distance import distance
import pandas as pd #basics
import numpy as np
import warnings,joblib,time,random,re,os,shutil
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt #plotting
from matplotlib.colors import LinearSegmentedColormap
from datetime import datetime
from joblib import Parallel, delayed #parallel processing
from scipy.stats import gaussian_kde #regression
from scipy.odr import ODR, Model, RealData
from sklearn.linear_model import LinearRegression,TheilSenRegressor
ee.Authenticate()
ee.Initialize()

## Variables
These Variables are user defined and can be used to customize the output of the tool

In [2]:
###Define Image collections###
LS8 = "LANDSAT/LC08/C02/T1_L2"
LS7 = "LANDSAT/LE07/C02/T1_L2"

#LS8 = "LANDSAT/LC08/C02/T1_TOA"
#LS7 = "LANDSAT/LE07/C02/T1_TOA"

harmonization_order=[LS7,LS8]
#an order of [LS7,LS8] means you will get an equation for making LS7 look like LS8

Aoi_shp_path= r'E:\GIS\CFS_FFF\AOIs\FFF_AOI.shp'
#AOI shapefile for the region you want to create normalization coeficients for
#Perfomance might be lower on AOI's that cover less than 10 WRS tiles

Load_sample_points=False
sample_points_path=r'E:\GIS\Landsat Normalization\Paper_results\Sample_points\1000000_sample_points.shp'

wrs_shp_path=r'E:\GIS\Landsat Normalization\Landsat_WRS_index\WRS_overlaps.shp'
#This is the link to WRS_overlaps.shp. It is necessary for finding overlaps

Save_data=True
Save_folder_path=r'E:\GIS\Lichen_work_master\Harmonization\QCLBSR50cc_m78_ally_deep_no_watersnow'

months=[7,8]#needs to be a list of months
#[1,2,3,4,5,6,7,8,9,10,11,12] is full range of months
years=[2013,2014,2015,2016,2017,2018,2019,2020,2021,2022]#needs to be a list of years
#[2013,2014,2015,2016,2017,2018,2019,2020,2021,2022] is full range of years

maxCloudCover=50 #maximum cloud coverage per Landsat image

CFMask_filtering=True #used to enable CFMask filtering
Water= False #if you want to include Water Pixels (CFMask), Set to True (yes) or False (no)
Snow= False #if you want to include Snow pixels (CFMask), Set to True (yes) or False (no)

Max_img_samples=10 #max number of images sampled between LS7 and LS8 overlap

Pixel_difference=1 #discarding pixels that are 100% different (same EQ as Roy et al)
#this is the maximum difference between LS7 and LS8 pixels allowed
#if the scatterplots look weird, or there is smoke in imagery, set to a lower value like .5 for 50% different

sample_points_n=1000000 #max is 1,000,000 points
#number of random sample points to generate

Regression_types=["OLS","RMA","TheilSen"]
#["OLS","RMA","TheilSen"]

These functions are necessary for the script to work

In [3]:
num_cores=min(joblib.cpu_count(), 16)-2
def time_tracker(start_time):
    return "{}sec".format(round((datetime.now() - start_time).total_seconds(), 2))
def get_var_names_from_list(in_list):
    # Create a mapping from values to variable names
    value_to_name = {value: name for name, value in globals().items() if isinstance(value, str)}
    # Use this mapping to retrieve the variable names in the order of in_list
    return [value_to_name[value] for value in in_list if value in value_to_name]
def apply_correction_factors (LS8,LS7,gdf):
    if "TOA" in LS8 and "TOA" in LS7:
        return gdf
    elif "TOA" not in LS8 and "TOA" not in LS7:
        columns_to_transform=[col for col in gdf.columns if col not in ['L1_image_id','L2_image_id','geometry']]
        gdf[columns_to_transform]=gdf[columns_to_transform]*0.0000275 - 0.2
        return gdf
    else:
        print('ERROR!!! TOA and SR datasets selected')
def toa_or_sr (LS8,LS7):
    if "toa" in LS8.lower() and "toa" in LS7.lower():
        return "TOA"
    if "toa" in LS8.lower() or "toa" in LS7.lower():
        return "Non-Matching"
    else:
        return "SR"
def create_save_path():
    if Save_data==True: #remakes Save_folder_path
        if os.path.exists(Save_folder_path):
            shutil.rmtree(Save_folder_path)
        os.makedirs(Save_folder_path)

## Step 1. Create Overlap DF and Frequency Map
This step determines if the area of interest is too large to process as a whole in GEE, where if True it would be broken up into strips.Then, the AOI or strips of AOI are passed into GEE, and all the LS7 and LS8 image names that intersect and are within the time and cloud coverage queries are selected. String manipulation is used to get the path/row and date of each image. overlap_df is created by finding overlapes between LS7 and LS8 which are +/- 1 day and +/- 1 path. This is then grouped and counted to create frequency_gdf which shows the number of overlapping LS7 nand LS8 images.

In [4]:
def create_WRS_gdf(input_shp, WRS_shapefile):
    create_save_path()
    #this creates a gdf of only WRS tiles found in the shapefile
    full_AOI=gpd.read_file(input_shp)
    full_AOI=full_AOI.to_crs('EPSG:4326')  # Ensure CRS is set to EPSG:4326
    full_AOI['geometry']=full_AOI.geometry.buffer(0)
    wrsgdf=gpd.read_file(WRS_shapefile)
    wrsgdf=wrsgdf.to_crs('EPSG:4326')
    wrsgdf['geometry']=wrsgdf.geometry.buffer(0)
    overlap_wrs=gpd.sjoin(wrsgdf,full_AOI,how="inner",predicate='intersects')
    df_subset=overlap_wrs[['row1','path1','path2','geometry']].reset_index(drop=True)
    return df_subset,full_AOI
def split_aoi_into_strips(gdf, max_area=1e13):
    projected_gdf=gdf.to_crs('EPSG:8857')  # Project to equal area for calculation
    total_area=projected_gdf.geometry.area.sum()  # Calculate area
    if total_area<max_area:
        print('No need for splitting. Starting timer and proceeding.')
        return gdf, None
    minx,miny,maxx,maxy=gdf.total_bounds
    midy=(miny+maxy)/2  # Calculate the midpoint y-coordinate, make upper and lower strips
    upper_strip,lower_strip=box(minx,midy,maxx,maxy),box(minx,miny,maxx,midy)
    upper_gdf=gpd.overlay(gdf,gpd.GeoDataFrame(geometry=[upper_strip],crs='EPSG:4326'),how='intersection')
    lower_gdf=gpd.overlay(gdf,gpd.GeoDataFrame(geometry=[lower_strip],crs='EPSG:4326'),how='intersection')
    num_strips=int(np.ceil(total_area/max_area))# Determine the number of strips needed based on the area
    strips=[]
    strip_width=(maxx-minx)/num_strips 
    for i in range(num_strips):
        strip_minx=minx+i*strip_width
        strip_maxx=minx+(i+ 1)*strip_width
        upper_vertical_strip=box(strip_minx, midy, strip_maxx, maxy)# Process upper strip
        upper_strip_gdf=gpd.overlay(upper_gdf, gpd.GeoDataFrame(geometry=[upper_vertical_strip], crs='EPSG:4326'), how='intersection')
        strips.append(upper_strip_gdf)
        lower_vertical_strip=box(strip_minx, miny, strip_maxx, midy)# Process lower strip
        lower_strip_gdf=gpd.overlay(lower_gdf, gpd.GeoDataFrame(geometry=[lower_vertical_strip], crs='EPSG:4326'), how='intersection')
        strips.append(lower_strip_gdf)
    strips=[df for df in strips if not df.empty]
    print(f'{len(strips)} Strips created due to input AOI size. Starting timer and Proceeding.')
    return gdf, strips
def date_cloud_aoi_filter(Landsatcollection,month,year,aoi):
    #basic filters based on month, year, cloud coverage, and AOI
    return Landsatcollection\
    .filter(ee.Filter.calendarRange(month, month, 'month')).filter(ee.Filter.calendarRange(year, year, 'year')) \
    .filter(ee.Filter.lte('CLOUD_COVER',maxCloudCover))\
    .filterBounds(aoi)
def get_image_names(image_collection):
    image_ids=image_collection.aggregate_array('system:id').getInfo()
    return image_ids
def extract_date_path_row(image_id):
    #used to extract date,path,row from landsat image name
    date_str=image_id.split('_')[-1]
    row,path=int(image_id.split('_')[-2][3:6]),int(image_id.split('_')[-2][:3])
    return datetime.strptime(date_str,'%Y%m%d'),row,path
def process_names(image_names):
    name_list=[]
    df=None #process the names and make a df from list of image names
    for name in image_names:
        date, row, path=extract_date_path_row(name)
        detailed_name=[name,date, row, path]
        name_list.append(detailed_name)
        df=pd.DataFrame(name_list)
        df.columns=['id', 'date', 'row','path']
        df[['row', 'path']]=df[['row', 'path']].astype(int)
    if df is not None:
        return df
def cycle_through_image_names(LS1, LS2, aoi):
    aoi=geemap.geopandas_to_ee(aoi)
    matching_pairs=[]
    L1_df,L2_df=None, None
    for year in years:
        for month in months:
            L1_SR=date_cloud_aoi_filter(ee.ImageCollection(LS1),month,year,aoi)  # filter date, cloud, aoi
            L2_SR=date_cloud_aoi_filter(ee.ImageCollection(LS2),month,year,aoi)
            L1_names,L2_names=get_image_names(L1_SR),get_image_names(L2_SR)  # get names of all the images that are within the filter
            L1_df,L2_df=process_names(L1_names), process_names(L2_names)  # process the names using string manipulation and make into a df
            if L1_df is not None and not L1_df.empty and L2_df is not None and not L2_df.empty:  # not empty dfs or ones that do not exist
                for index_L1,row_L1 in L1_df.iterrows():
                    # Find rows in L2_df with the same 'row' and 'path' within +1 or -1, and date within 1 day
                    possible_1row_matches=L2_df[(L2_df['row']==row_L1['row']) &
                        ((L2_df['path']==row_L1['path'] + 1) | (L2_df['path']==row_L1['path'] - 1)) &
                        (abs(L2_df['date']-row_L1['date']).dt.days<= 1)]  # this is the formula for finding matches
                    for index_L2, row_L2 in possible_1row_matches.iterrows():
                        matching_pairs.append({'L1_id': row_L1['id'], 'L1_date': row_L1['date'], 'L1_row': row_L1['row'], 'L1_path': row_L1['path'],'L2_id': row_L2['id'], 'L2_date': row_L2['date'], 'L2_row': row_L2['row'], 'L2_path': row_L2['path']})
                    # Find rows in L2_df with same 'row' and edge paths within +1 or -1 date
                    possible_edge_matches=L2_df[(L2_df['row']==row_L1['row']) &
                        ((L2_df['path']==row_L1['path'] + 232) | (L2_df['path']==row_L1['path'] - 232)) &
                        (abs(L2_df['date']-row_L1['date']).dt.days<= 1)]  # this is the formula for finding matches
                    for index_L2, row_L2 in possible_edge_matches.iterrows():
                        matching_pairs.append({'L1_id': row_L1['id'], 'L1_date': row_L1['date'], 'L1_row': row_L1['row'], 'L1_path': row_L1['path'],'L2_id': row_L2['id'], 'L2_date': row_L2['date'], 'L2_row': row_L2['row'], 'L2_path': row_L2['path']})
    return matching_pairs
def chunk_list(lst,n):
    avg, out, last=len(lst) / float(n),[],0.0 #Divide a list into `n` chunks.
    while last<len(lst):
        out.append(lst[int(last):int(last + avg)])
        last+=avg
    return out
def process_strip_chunk(LS1,LS2,strips_chunk,chunk_index):
    all_matching_pairs=[]
    ee.Initialize()
    for strip_n,strip in enumerate(strips_chunk):
        strip_time=datetime.now()
        matching_pairs=cycle_through_image_names(LS1,LS2,strip)
        all_matching_pairs.extend(matching_pairs)
        matching_df=pd.DataFrame(all_matching_pairs).drop_duplicates(inplace=False)
    return all_matching_pairs
def saving_overlap_info(frequency_gdf,matching_df):
    dtype=toa_or_sr(LS8,LS7)
    col_names=get_var_names_from_list(harmonization_order)
    frequency_gdf.to_file(f'{Save_folder_path}\\frequencey_gdf.shp', driver='ESRI Shapefile')
    frequency_gdf.plot(column='count_gee',cmap='viridis', legend=False)
    plt.title(f'Frequency of {dtype} {col_names[0]} and {col_names[1]} Overlaps', fontsize=16)
    plt.savefig(f'{Save_folder_path}\\frequencey_plot.png',dpi=500)
    matching_df.to_csv(f'{Save_folder_path}\\overlap.csv')
def create_overlap_df(harmonization_order, aoi, strips):
    print(f'{toa_or_sr(LS8,LS7)} imagery datasets')#used to print to user what imagery dataset they are using
    LS1, LS2=ee.ImageCollection(harmonization_order[0]),ee.ImageCollection(harmonization_order[1])#order of harmonization
    start_time=datetime.now()
    if strips is None:
        matching_pairs=cycle_through_image_names(LS1, LS2, aoi)
        overlap_df=pd.DataFrame(matching_pairs)
    else:
        print(f'Processing {len(strips)} strips')
        chunks=chunk_list(strips, num_cores)# Split the strips list into smaller chunks for parallel processing
        results=Parallel(n_jobs=num_cores)(
            delayed(process_strip_chunk)(LS1, LS2, chunk, idx) for idx, chunk in enumerate(chunks))
        all_matching_pairs=[pair for result in results for pair in result]# Combine results from all chunks
        overlap_df=pd.DataFrame(all_matching_pairs)
        overlap_df.drop_duplicates(inplace=True)
    print(f"GEE filtering completed in {time_tracker(start_time)}")
    if overlap_df.empty:
        print("No matches available")
        return None, None
    grouped_df=overlap_df.groupby(['L1_row','L1_path','L2_path']).size().reset_index(name='count_gee')#create frequency_GDF
    frequency_gdf=pd.merge(grouped_df, WRS_gdf, left_on=['L1_row','L1_path','L2_path'], right_on=['row1','path1','path2'], how='inner')
    if frequency_gdf.empty:
        print("No matches available")
        return None, None
    frequency_gdf=gpd.GeoDataFrame(frequency_gdf,geometry='geometry')
    frequency_gdf.crs='EPSG:4326'
    frequency_gdf = frequency_gdf.drop_duplicates(subset='geometry')#get rid of duplicates
    if Save_data==True:
        saving_overlap_info(frequency_gdf,overlap_df)
    return overlap_df,frequency_gdf

## Step 2. Create Sample Points within AOI
This sections projects the AOI into an equal earth projection and randomly generates sample_points_n points within it. For points that do not land within a LS7/LS8 overlap, the point is moved to the nearest overlap and then randomly assigned a location within the polygon. The output (overlap_points_gdf) contains all the points that will be sampled in Step 3. 

In [5]:
def generate_equalA_points (AOI,points_n):
    AOI=AOI.to_crs('EPSG:8857')#make into equal earth projection for even global sampling
    transform=rasterio.transform.from_bounds(*AOI.total_bounds,10000,10000)#creates a raster grid
    mask=geometry_mask([geom for geom in AOI.geometry], transform=transform, invert=True, out_shape=(10000,10000))#creates masked raster grid
    rows, cols=np.where(mask)#Sample points from the mask raster grid
    random_indices=np.random.choice(len(rows), size=points_n, replace=False)
    sampled_rows,sampled_cols=rows[random_indices],cols[random_indices]
    x_coords, y_coords=rasterio.transform.xy(transform, sampled_rows, sampled_cols)
    points=[Point(x, y) for x, y in zip(x_coords, y_coords)]#Create points from the sampled coordinates
    points_gdf=gpd.GeoDataFrame(geometry=points, crs='EPSG:8857')
    print('Initial points generated, ensuring they are in WRS overlaps')
    return points_gdf
def find_nearest_polygon_and_generate_point(WRS_gdf, point):
    point = gpd.GeoSeries([point], crs=WRS_gdf.crs)#Ensure the point is in the same CRS as the polygons and AOI
    WRS_gdf['distance']=WRS_gdf['geometry'].distance(point.iloc[0])#Calculate the distance from the point to each polygon in WRS_gdf
    nearest_polygon=WRS_gdf.loc[WRS_gdf['distance'].idxmin()]['geometry']#Find the polygon with the minimum distance
    minx, miny, maxx, maxy=nearest_polygon.bounds #Generate a random point within the intersection polygon
    while True:
        random_point=Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
        if nearest_polygon.contains(random_point):
            return random_point
def process_points(aoi_points,overlap_gdf):
    start_time=datetime.now()
    overlap_gdf=overlap_gdf.to_crs('EPSG:8857')#Project to equal earth
    new_points=[]
    AOI_array=np.array([(point.x, point.y) for point in aoi_points.geometry])#Make AOI array for distance calculations
    for point_array in AOI_array:
        point=Point(point_array[0], point_array[1])
        if overlap_gdf.contains(point).any():
            new_points.append(point)#Append point if it is inside a WRS tile
        else:
            alt_point=find_nearest_polygon_and_generate_point(overlap_gdf, point)#If not, find closest alternative
            new_points.append(alt_point)#Append the alt point to the list directly
    return gpd.GeoDataFrame(geometry=new_points, crs='EPSG:8857')
def parallel_process_points(aoi_points,frequency_gdf,full_AOI):
    start_time=datetime.now()
    full_AOI=full_AOI.to_crs('EPSG:8857')
    frequency_gdf=frequency_gdf.to_crs('EPSG:8857')
    clip_gdf=gpd.clip(frequency_gdf, full_AOI) #clipping frequency_gdf to AOI
    clip_gdf=clip_gdf.to_crs('EPSG:8857') # making it equal area
    clip_gdf=clip_gdf.explode().reset_index(drop=True) #removing multipolygons
    point_count=len(aoi_points)
    subset_size=max(point_count // num_cores, 1)
    aoi_subsets=[aoi_points.iloc[i:i + subset_size] for i in range(0, point_count, subset_size)]
    results=Parallel(n_jobs=num_cores)(
        delayed(process_points)(aoi_subset, clip_gdf) for aoi_subset in aoi_subsets)
    combined_gdf=gpd.GeoDataFrame(pd.concat(results, ignore_index=True))
    combined_gdf=combined_gdf.to_crs('EPSG:4326')#Convert back to WGS 84
    if Save_data==True:
        combined_gdf.to_file(f'{Save_folder_path}\\{sample_points_n}_sample_points.shp', driver='ESRI Shapefile')
    print(f'Processed in {time_tracker(start_time)}')
    return combined_gdf
def create_overlap_points_gdf(points_gdf,wrs_gdf):
    points_gdf=points_gdf.to_crs(wrs_gdf.crs) #appends path/row information to points
    joined_gdf=gpd.overlay(points_gdf, wrs_gdf[['geometry','L1_row','L1_path','L2_path']], how='intersection')# Perform the spatial join using 'overlap'
    joined_gdf=joined_gdf[['L1_row','L1_path','L2_path','geometry']].drop_duplicates(subset='geometry') # Clean up the resulting GeoDataFrame
    return joined_gdf.reset_index(drop=True).drop_duplicates()
def run_workflow(full_AOI,sample_points_n,frequency_gdf,Load_sample_points,sample_points_path):
    if Load_sample_points==True:
        Sample_points_gdf = gpd.read_file(sample_points_path)
        overlap_points_gdf=create_overlap_points_gdf(Sample_points_gdf,frequency_gdf)
        print('Loaded sample points')
    else:
        AOI_points=generate_equalA_points(full_AOI,sample_points_n)#makes random points in equal earth prj
        Sample_points_gdf=parallel_process_points(AOI_points,frequency_gdf,full_AOI)#moves points inside WRS tiles
        overlap_points_gdf=create_overlap_points_gdf(Sample_points_gdf,frequency_gdf)#appends path/row information to points
    return Sample_points_gdf,overlap_points_gdf

## Step 3. Create GDF containing pixel values
The objective of this part of the script is to use overlap_points_gdf to sample pixel values found LS7 and LS8 Gee imagery. Max_img_samples is the number of image pairs are selected for each LS7/LS8 overlap. Once a point has a clear observation in both LS7 and LS8 images (and complies with Pixel_difference between LS7/LS8), it is added to big_gdf and removed from further sampling. The output from this section, big_gdf, contains the pixel values for both LS7 and LS8 sensors, the image names, and the location of the point that was sampled. 